In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from keras.models import load_model
import os 


with open('../saved_model/ens_model.pkl', 'rb') as file:
    ens_model = pickle.load(file)
with open('../saved_model/knn_model.pkl', 'rb') as file:
    knn_model = pickle.load(file)
with open('../saved_model/svm_model.pkl', 'rb') as file:
    svm_model = pickle.load(file)
nn_model = load_model('../saved_model/nn_model.keras')  # Assuming load_model loads the neural network


2024-05-20 14:51:41.918965: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-20 14:51:41.961520: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 14:51:42.766711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('/home/khangpt/MUSIC-GEN-PROJ/GTZAN/Data/features_30_sec.csv')
def preprocess_(df):
    df = df.iloc[0:,1:]
    X = df.drop(['length','label'], axis = 1)
    y = df['label']

    df.label = pd.Categorical(df.label)
    y = np.array(df.label.cat.codes)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    return X, y
X, y = preprocess_(df)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

In [3]:
y_pred =  nn_model.predict(X_test).argmax(axis=-1)
print(accuracy_score(y_test,y_pred))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
0.76


In [4]:
y_pred =  svm_model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.735


In [5]:
y_pred =  knn_model.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.745


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input
from scikeras.wrappers import KerasClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

# Define the KNN classifier
KNN = KNeighborsClassifier(n_neighbors=6, weights='distance')

# Define the SVM classifier
SVM = SVC(kernel='rbf', gamma='scale', C=11, probability=True)

# Define the neural network model
def create_nn():
    model = Sequential([
        Input(shape=(57,)), 
        Dense(300, input_dim=57,activation='relu'),                                                                      
        Dense(300, activation='relu'),
        Dense(300, activation='relu'),
        Dense(10, activation='softmax')
    ])
    adam = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=adam,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

NN = KerasClassifier(build_fn=create_nn, epochs=150, batch_size=32, verbose=0)

In [10]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

base_models = [('knn', KNN), ('svm', SVM), ('nn', NN)]
stack = StackingClassifier(estimators=base_models,
                              final_estimator=LogisticRegression())
restack = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    passthrough=True,
)

models = base_models + [('stack',stack),('restack',restack)]


In [8]:
import matplotlib.pyplot as plt

In [12]:
restack.fit(X_train,y_train)
y_pred_restack = restack.predict(X_test)
print(accuracy_score(y_pred_restack,y_test))


/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/khangpt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/khangpt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to 

0.765


In [ ]:

print(classification_report(y_test, y_pred_restack))

NameError: name 'classification_report' is not defined